In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import urllib.request
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import urllib
import socket, whois


###  total_count , total_avg , max_len_string , above_avg_count

In [2]:
def long_strings_count(url) :
    options = Options()
    options.add_argument("--headless")
    options.add_argument('/home/archit/sem2/ai/chromedriver/chromedriver')

    browser = webdriver.Chrome('/home/archit/sem2/ai/chromedriver/chromedriver', options=options)
    browser.get("http://" + url)
    soup = BeautifulSoup(browser.page_source, "html.parser")
    scripts = soup.findAll('script')
    
    count = 0
    total_avg = 0
    total_count = 0 
    max_len_string = 0
    above_avg_count =  0 
    
    for sc in scripts:
        check = False
        try :
            src = sc['src']
        except :
            src = None 
        
        if src is None :
            txt = sc.text
        else :
            try :
                txt = urlopen(src).read().decode('utf-8')
            except :
                txt = None
                check = True
        
        if txt is None : 
            continue 

        result1 = re.findall(r'"(.*?)"', txt)
        result2 = re.findall(r"'(.*?)'", txt)
        
        result = result1 + result2
        
        try :
            temp_avg = sum( map(len, result)) 
            temp_count =  len(result)
            max_len_string = max( max_len_string , len(max(result, key=len)))
        except :
            temp_avg = 0
            temp_count =  0
            max_len_string = max_len_string
        
        total_avg += temp_avg
        total_count  += temp_count
        
  
        
        try : 
            for ele in result :
                if len(ele) > temp_avg/temp_count :
                    above_avg_count += 1
        except :
            above_avg_count = above_avg_count
        
    return total_count , total_avg , max_len_string , above_avg_count
        

### Scripts count  , Iframe count  ,external url count 

In [30]:
def find_list_resources(tag, attribute,soup):
    list = []
    for x in soup.findAll(tag):
        try:
            list.append(x[attribute])
        except KeyError:
            pass
    return(list)


def iframe_count(url):
    url = "http://" + url
    try :
        response = requests.get(url)
    except : 
        response = None
        
    if response is None :
        return 0 
    
    soup = BeautifulSoup(response.text, "html.parser")
    return len(soup.findAll('iframe'))
    
    
        
def scripts_count(url) :
    url  = "http://" + url
    try :
        response = requests.get(url)
    except : 
        response = None
        
    if response is None :
        return 0
    
    soup = BeautifulSoup(response.text, "html.parser")
    return len(soup.findAll('script'))


def external_url_count(url) :
    url =  "http://" + url
    try :
        response = requests.get(url)
    except : 
        response = None
    
    if response is None :
        return 0
    
    soup = BeautifulSoup(response.text, "html.parser")
    count = 0
    count += len(find_list_resources('img',"src",soup))   
    count += len(find_list_resources('script',"src",soup))    
    count += len(find_list_resources("link","href",soup))
    count += len(find_list_resources("video","src",soup))         
    count += len(find_list_resources("audio","src",soup)) 
    count += len(find_list_resources("iframe","src",soup))
    count += len(find_list_resources("embed","src",soup))
    count += len(find_list_resources("object","data",soup))         
    count += len(find_list_resources("source","src",soup))
    return count

### Having ip address 

In [10]:
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

### https  token   

In [11]:
def https_token(url):
    match = re.search('https://|http://', url)
    if match==None:
        return 0
    if match.start(0) == 0:
        url = url[match.end(0):]
    match = re.search('http|https', url)
    if match:
        return 0
    else:
        return 1

### DNS response 

In [12]:
import socket
import requests as req
import re
import time

def dns_response(url):
    try :
        url = url.split("/")
        hostname = url[0]
    except : 
        print(" issue in finding host name from url : " + url)
        return -2
    
    try :
        startTime = time.time()
        ip = socket.gethostbyname(hostname)
        endTime = time.time()
        diff = (endTime - startTime)* 1000
        return diff
    except :
        return -1

### handshake_time, data_time, data_length , redirect , protocol , http_status_code,server_name

In [13]:
from urllib.parse import urlparse
import time
import ssl
import urllib.request  as urllib2
import urllib

def networks(link):
    if not urlparse(link).scheme:
        url = "http://" + link
    gcontext = ssl._create_unverified_context()
    handshake_start = time.time()
    http_status_code = 400

    try : 
        stream = urllib2.urlopen(url,context=gcontext)
        http_status_code = stream.getcode()

    except urllib.error.HTTPError  as e :
            http_status_code = e.code
    
    
    if http_status_code >= 400 :
        return 0 , 0 , 0 , False , 'http' , http_status_code , None      
            
        
    new_url = stream.geturl()
    
    try : 
        server_name = stream.info()["Server"]
    except :
        server_name = None
    
    redirect = False
    protocol =  "http"
    
    if new_url :
        new_url = new_url.split("://")
        protocol =  new_url[0]
        del new_url[0]
        new_url = "://".join(new_url)
        
        if new_url != link:
            redirect = True

            
        
#     print(stream.info())
    handshake_end = time.time()
    handshake_time = (handshake_end - handshake_start) * 1000
#     print ('HTTP handshake time = %.5f ms' %handshake_time)
    data_start = time.time()
    data_length = len(stream.read())
    data_end = time.time()
    data_time = (data_end - data_start)*1000
#     print("data time = %.5f ms" %data_time)
#     print(data_length)
    return handshake_time, data_time, data_length , redirect , protocol , http_status_code,server_name
# networks("twitter.com/aQuariusrecOrds")

### whois data 

In [14]:
import whois
import os
import subprocess
import re

In [15]:
def whois_features(url):
    url = url.split("/")[0]

    try :
        batcmd= "whois " + url 
        result = subprocess.check_output(batcmd, shell=True)
        result = result.decode("utf-8")
        
        cd1 = re.findall('Creation Date:\s.*\n', result, re.M)
        cd2 = re.findall('created:\s.*\n', result, re.M)
        
        if len(cd1) >= 1 :
            c_date = cd1[0]
        elif len(cd2) >=1 :
            c_date = cd2[0]
        else :
            c_date  = "cdate_not_available"
        
        if c_date != "cdate_not_available" :
            c_date = c_date.split(": ")[1].strip()
        
        
        rd2 = re.findall('expires.*\n', result, re.M)
        rd1 = re.findall('Registry Expiry Date:\s.*\n', result, re.M)
        if len(rd1) >= 1 :
            r_date = rd1[0]
        elif len(rd2) >=1 :
            r_date = rd2[0]
        else :
            r_date  = "rdate_not_available"
        if r_date != "rdate_not_available" :
            r_date = r_date.split(": ")[1].strip()
        

        
        ud1 = re.findall('Updated Date:\s.*\n', result, re.M)
        ud2 = re.findall('.*Updated\s.*\n', result, re.M)
        ud3 = re.findall('last-modified:\s.*\n', result, re.M)
        if len(ud1) >= 1 :
            u_date = ud1[0]
        elif len(ud2) >=1 :
            u_date = ud2[0]
        elif len(ud3) >= 1:
            u_date = ud3[0]
        else :
            u_date  = "udate_not_available"
        
        if u_date != "udate_not_available" :
            u_date = u_date.split(": ")[1].strip()
        
        con1 = re.findall('Admin Country:\s.*\n', result, re.M)
        con2 = re.findall('[c,C]ountry:\s.*\n', result, re.M)
        if len(con1) >= 1 :
            con = con1[0]
        elif len(rd2) >=1 :
            con = con2[0]
        else :
            con  = "country_not_available"
        if con != "country_not_available":
            con  = con.split(": ")[1].strip()
            
            
                
        st1 = re.findall('Admin State/Province:\s.*\n', result, re.M)
        st2 = re.findall('City:\s.*\n', result, re.M)
        if len(st1) >= 1 :
            st = st1[0]
        elif len(st2) >=1 :
            st = st2[0]
        else :
            st  = "state_not_available"
        if st !=  "state_not_available" :
            st = st.split(": ")[1].strip()
            
    except:
        c_date = "cdate_not_available"
        r_date = "rdate_not_available"
        u_date = "udate_not_available"
        con = "country_not_available"
        st  = "state_not_available"

        
    return c_date , r_date , u_date , con , st

# Finding data features 

In [16]:
def find_features(url_arr):
    result  = []
    for url in url_arr:
        di = {}
        di["url_length"] = len(url)
        di['special_character_count'] = len(re.sub('[\w]+' ,'', url))
        di['having_ip_address'] = having_ip_address(url)
        
        res = dns_response(url)
        if res == -1 :
            print("Error : "+ url + " , No DNS info found.")
            continue
        di['dns_response_time'] = res
        
        
        di['iframe_count']  =  iframe_count(url)
        di['scripts_count']  =  scripts_count(url)        
        di['external_link_count']  =  external_url_count(url)
        
        
        try : 
            handshake_time, data_time, data_length , redirect , protocol , http_status_code,server_name = networks(url)
        except :
            handshake_time, data_time, data_length , redirect , protocol , http_status_code,server_name = None,None,None,None,None, None,None
            
        di['handshake_time'] = handshake_time 
        di['data_time'] = data_time
        di['data_length'] = data_length
        di['redirect'] = redirect
        di['protocol'] = protocol
        di['http_status_code'] = http_status_code
        di['server_name'] = server_name
        
        
        try :
            total_count , total_avg , max_len_string , above_avg_count = long_strings_count(url)
        except :
            total_count , total_avg , max_len_string , above_avg_count = None, None, None, None
        di["total_strings_count"] = total_count
        di["average_string_length"] = total_avg
        di["max_len_strings"] = max_len_string
        di["strings_above_avglength"] = above_avg_count
        
        
        c_date , r_date , u_date , con , st = whois_features(url)
        di["whois_creation_date"] = c_date
        di["whois_expiry_date"] = r_date
        di["whois_last_updated"] = u_date
        di["whois_country"] = con
        di["whois_city"] = st
        
        result.append(di)
    
    return result
           
        

In [32]:
find_features(["twitter.com/aQuariusrecOrds"])

[{'url_length': 27,
  'special_character_count': 2,
  'having_ip_address': 0,
  'dns_response_time': 2.5920867919921875,
  'iframe_count': 6,
  'scripts_count': 8,
  'external_link_count': 73,
  'handshake_time': 701.6332149505615,
  'data_time': 359.483003616333,
  'data_length': 313049,
  'redirect': False,
  'protocol': 'https',
  'http_status_code': 200,
  'server_name': 'tsa_k',
  'total_strings_count': 26397,
  'average_string_length': 1663615,
  'max_len_strings': 46438,
  'strings_above_avglength': 2436,
  'whois_creation_date': 'cdate_not_available',
  'whois_expiry_date': 'rdate_not_available',
  'whois_last_updated': 'udate_not_available',
  'whois_country': 'country_not_available',
  'whois_city': 'state_not_available'}]